# Regression mit Holdout und anderen Regressoren

In diesem Notebook wirst du mit dem Holdout-Verfahren und multidimensionaler Regression arbeiten. Außerdem wirst du dir anschauen, wie gut andere Regressoren funktionieren.

## Datenset laden

Dieses Vorgehen kennst du schon:

In [ ]:
from sklearn import datasets
california = datasets.fetch_california_housing()

Wie gehabt wandelst du das Datenset in einen `DataFrame`.

In [ ]:
import pandas as pd
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Price"] = california.target

## Lineare Regression mit Trainings- und Test-Daten

Du kennst das *Holdout-Verfahren* schon von der Klassifikation. Hier kannst du es genauso anwenden:

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(df[["MedInc"]].values, df["Price"].values)

Die Regression `fit`test du nur mit den Trainingsdaten

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

## Fehler berechnen

In diesem Fall reicht es, wenn du einen Score berechnest. Nutze den `r2`-Score:

In [ ]:
from sklearn import metrics
metrics.r2_score(y_train, lr.predict(X_train))

Wie sieht der Score bei den Testdaten aus?

In [ ]:
metrics.r2_score(y_test, lr.predict(X_test))

Sogar besser - von *Overfitting* kann hier also keine Rede sein!

## Mehrdimensionale Regression

Mit `sklearn` kannst du auch mehrdimensionale (oder sog. *multivariate*) Regression durchführen. Es ist gar nicht einfach, die richtigen Features zu finden. In diesem Beispiel benutzt du einfach *alle*:

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(california.data, california.target)

Der Aufruf unterscheidet sich nicht:

In [ ]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

Überprüfe den Score:

In [ ]:
metrics.r2_score(y_train, mlr.predict(X_train))

Das ist viel besser als mit einem Feature! Allerdings liegt das zum Teil auch daran, dass der `r2`-Wert immer besser wird, wenn du mehr Features verwendest. Um die Qualität zu beurteilen, benötigst du den *justierten* `r2`:

In [ ]:
X_train.shape

In [ ]:
def adjusted_r2(r2, n, k):
    return 1 - ((1-r2)*(n-1)/(n-k-1))

adjusted_r2(metrics.r2_score(y_train, mlr.predict(X_train)), *X_train.shape)

Da du nicht viele Features verwendest im Vergleich zu der Anzahl der Samples, ist der Unterschied sehr gering.

Wie sieht es beim Testdatenset aus?

In [ ]:
adjusted_r2(metrics.r2_score(y_test, mlr.predict(X_test)), *X_test.shape)

Auch besser! Das sieht auch hier nicht nach Overfitting aus.

Wenn du möchtest, kannst du `sklearn` ausrechnen lassen, welche Features die wichtigsten sind. Dazu dient die Funktion `permutation_importance':

In [ ]:
from sklearn import inspection
pi = inspection.permutation_importance(mlr, X_train, y_train)
pi

Etwas übersichtlicher kannst du das in einem `DataFrame` darstellen:

In [ ]:
idf = pd.DataFrame()
idf["feature"] = california.feature_names
idf["importance"] = pi.importances_mean
idf

In diesem Fall sind `Latitude` und `Longitude` sogar noch wichtigere Features. Bei einem genügend großen Datenset definieren diese die Lage der Objekte, die ganz offensichtlich eine Auswirkung auf den Preis hat.

## Unterschiedliche Regressoren

Neben der linearen Regression gibt es noch andere Regressoren, die du ausprobieren kannst. Du lässt dir immer den `r2` für Trainings- und Testdaten ausgeben, um den Regressor beurteilen zu können.

In [ ]:
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.ensemble import ExtraTreesRegressor
for r in [LinearRegression(), SGDRegressor(loss='squared_error', penalty='l2',  random_state=42), 
          Ridge(), ExtraTreesRegressor(n_estimators=10,random_state=42)]:
    print(r)
    r.fit(X_train, y_train)
    print(metrics.r2_score(y_train, r.predict(X_train)), 
          metrics.r2_score(y_test, r.predict(X_test)))
          

`ExtraTreesRegressor` ist eine *Mischung* verschiedener Ensemble-Verfahren. Obwohl die Traingsdaten überbestimmt scheinen, ist die Performance auch in den Testdaten sehr gut.

Der `ExtraTreesRegressor` kann direkt die wichtigsten Features ausgeben:

In [ ]:
idf = pd.DataFrame()
idf["feature"] = california.feature_names
idf["importance"] = r.feature_importances_
idf

Auch hier siehst du, dass neben dem Einkommen auch die Lage eine Rolle spielt.

## Overfitting bei Regression

Wie bei Klassifikatoren gibt es auch bei Regressoren *Overfitting*. Um das beurteilen zu können, musst du die Daten in ein Trainings- und Testdatenset splitten.

Im Gegensatz zur Klassifikation kannst du hier mit einem Score arbeiten, was die Beurteilung insgesamt einfacher macht.